# API Foursquare.

Una vez seleccionada la oficina perfecta que cumple nuestras necesidades vamos a comprobar mediante la API de Foursquare los siguientes puntos;
- los Account managers viajan mucho. Voy a busar un aeropuerto.
- El 30% de los empleados de la compañía tiene al menos un niño. Voy a buscar guarderías.
- La edad de los trabajadores oscila entre 25 y 40 años. Voy a buscar discotecas.
- A los ejecutivos les gusta mucho la cadena de cafés "Starbucks". Voy a buscar Starbucks.
- La CEO de la compañía es vegana. Voy a buscar restaurantes veganos.

In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd

In [2]:
load_dotenv()

True

In [3]:
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

In [4]:
suomi = pd.read_csv('../data/dataOK.csv', encoding = 'ISO - 8859-1')
suomi

,city,country_code,latitude,longitude,name,category_code,number_of_employees,founded_year
0,Helsinki,FIN,60.171821,24.945167,Apaja,games_video,44,2001


In [5]:
#Busco los códigos en la API en los que voy a basar las llamadas;

"""
airport = "4bf58dd8d48988d1ed931735"

nurseries = "4f4533814b9074f6e4fb0107"

nightlife sport = "4d4b7105d754a06376d81259"

coffee_shop = "4bf58dd8d48988d1e0931735"

vegan_restaurant = "4bf58dd8d48988d1d3941735"

"""

'\nairport = "4bf58dd8d48988d1ed931735"\n\nnurseries = "4f4533814b9074f6e4fb0107"\n\nnightlife sport = "4d4b7105d754a06376d81259"\n\ncoffee_shop = "4bf58dd8d48988d1e0931735"\n\nvegan_restaurant = "4bf58dd8d48988d1d3941735"\n\n'

#  Apaja.

## Primera llamada: Aeropuerto

In [6]:
donde = "21 Mikonkatu, Helsinki FI 00100" #Empresa 1: Apaja
def geocode(address):
    """
    Saca las coordenadas de una dirección que le des.
    """
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    try:
        return {
            "type":"Point",
            "coordinates":[float(data["longt"]),float(data["latt"])]}
    except:
        return data

In [7]:
emp1 = geocode(donde)
emp1

{'type': 'Point', 'coordinates': [24.94519, 60.17212]}

In [8]:
url1= 'https://api.foursquare.com/v2/venues/explore'

In [9]:
params = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{emp1.get('coordinates')[1]},{emp1.get('coordinates')[0]}",
              "query": "Airport",
              "limit": 1
             }

In [10]:
resp = requests.get(url=url1, params=params)
airdata = json.loads(resp.text)


In [11]:
airdata.keys()

dict_keys(['meta', 'response'])

In [12]:
decoding_data = airdata.get("response")

In [13]:
decoded = decoding_data.get("groups")[0]

In [14]:
airp = decoded.get("items")

In [15]:
for k,v in airp[0].items():
    print(k,"-->",v,"\n")

reasons --> {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]} 

venue --> {'id': '4adcdb27f964a520dd6121e3', 'name': 'Helsinki Airport (HEL) (Helsinki-Vantaan lentoasema)', 'contact': {}, 'location': {'address': 'Lentäjänkuja 1', 'crossStreet': 'Lentoasemantie', 'lat': 60.31912855684579, 'lng': 24.968297481536865, 'labeledLatLngs': [{'label': 'display', 'lat': 60.31912855684579, 'lng': 24.968297481536865}], 'distance': 16414, 'postalCode': '01530', 'cc': 'FI', 'neighborhood': 'Lentokenttä', 'city': 'Vantaa', 'state': 'Uusimaa', 'country': 'Suomi', 'formattedAddress': ['Lentäjänkuja 1 (Lentoasemantie)', '01530 Vantaa', 'Suomi']}, 'categories': [{'id': '4bf58dd8d48988d1ed931735', 'name': 'Airport', 'pluralName': 'Airports', 'shortName': 'Airport', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/airport_', 'suffix': '.png'}, 'primary': True}], 'verified': False, 'stats': {'tipCount': 0, 'usersCount': 

In [16]:
from functools import reduce
import operator

In [17]:
mapa_nombre = ["venue","name"]
m_latitud = ["venue","location","lat"]
m_longitud = ["venue","location","lng"]

In [18]:
def getFromDict(diccionario,mapa):
    return reduce (operator.getitem,mapa,diccionario)

In [19]:
print(getFromDict(airp[0],mapa_nombre))

Helsinki Airport (HEL) (Helsinki-Vantaan lentoasema)


In [20]:
print(getFromDict(airp[0],m_latitud))

60.31912855684579


In [21]:
print(getFromDict(airp[0],m_longitud))

24.968297481536865


In [22]:
lovemosclaro = []
for dic in airp:
    paralista = {}
    paralista["name"] = getFromDict(dic,mapa_nombre)
    paralista["latitud"] = getFromDict(dic,m_latitud)
    paralista["longitud"] = getFromDict(dic,m_longitud)
    lovemosclaro.append(paralista)

In [23]:
lovemosclaro[0]

{'name': 'Helsinki Airport (HEL) (Helsinki-Vantaan lentoasema)',
 'latitud': 60.31912855684579,
 'longitud': 24.968297481536865}

In [24]:
dataframefinal = pd.DataFrame(lovemosclaro)
dataframefinal.head()

,name,latitud,longitud
0,Helsinki Airport (HEL) (Helsinki-Vantaan lento...,60.319129,24.968297


In [25]:
import geopandas as gpd

In [26]:
gdf = gpd.GeoDataFrame(dataframefinal, geometry = gpd.points_from_xy(dataframefinal.longitud,dataframefinal.latitud))
gdf.head()

,name,latitud,longitud,geometry
0,Helsinki Airport (HEL) (Helsinki-Vantaan lento...,60.319129,24.968297,POINT (24.96830 60.31913)


In [27]:
from cartoframes.viz import Map, Layer, popup_element


In [28]:
Map(Layer(gdf, popup_hover = [popup_element("name","Airport")]))


## Segunda llamada: Nurseries

In [29]:
params = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{emp1.get('coordinates')[1]},{emp1.get('coordinates')[0]}",
              "query": "Nursery School",
              "limit": 5
             }

In [30]:
resp = requests.get(url=url1, params=params)
nurdata = json.loads(resp.text)

In [31]:
nurdata.keys()

dict_keys(['meta', 'response'])

In [32]:
decoding_data = nurdata.get("response")

In [33]:
decoded = decoding_data.get("groups")[0]

In [34]:
nur = decoded.get("items")


In [35]:
for k,v in nur[0].items():
    print(k,"-->",v,"\n")

reasons --> {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]} 

venue --> {'id': '5ca61d09a2a6ce002c790dad', 'name': 'Ankkalampi Kruununhaka', 'contact': {}, 'location': {'address': 'Rauhankatu 10', 'lat': 60.171618, 'lng': 24.955742, 'labeledLatLngs': [{'label': 'display', 'lat': 60.171618, 'lng': 24.955742}], 'distance': 586, 'postalCode': '00170', 'cc': 'FI', 'city': 'Helsinki', 'state': 'Uusimaa', 'country': 'Suomi', 'formattedAddress': ['Rauhankatu 10', '00170 Helsinki', 'Suomi']}, 'categories': [{'id': '4f4533814b9074f6e4fb0107', 'name': 'Nursery School', 'pluralName': 'Nursery Schools', 'shortName': 'Nursery School', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_', 'suffix': '.png'}, 'primary': True}], 'verified': False, 'stats': {'tipCount': 0, 'usersCount': 0, 'checkinsCount': 0, 'visitsCount': 0}, 'beenHere': {'count': 0, 'lastCheckinExpiredAt': 0, 'marked': False, 'unconfirmed

In [36]:
from functools import reduce
import operator

In [37]:
mapan_nombre = ["venue","name"]
mn_latitud = ["venue","location","lat"]
mn_longitud = ["venue","location","lng"]

In [38]:
def getFromDict(diccionario,mapa):
    return reduce (operator.getitem,mapa,diccionario)

In [39]:
print(getFromDict(nur[0],mapan_nombre))
print(getFromDict(nur[0],mn_latitud))
print(getFromDict(nur[0],mn_longitud))

Ankkalampi Kruununhaka
60.171618
24.955742


In [40]:
nurseriesF = []
for dic in nur:
    paralista = {}
    paralista["name"] = getFromDict(dic,mapa_nombre)
    paralista["latitud"] = getFromDict(dic,m_latitud)
    paralista["longitud"] = getFromDict(dic,m_longitud)
    nurseriesF.append(paralista)

In [41]:
nurseriesF[0]

{'name': 'Ankkalampi Kruununhaka', 'latitud': 60.171618, 'longitud': 24.955742}

In [42]:
df_nur = pd.DataFrame(nurseriesF)
df_nur.head()

,name,latitud,longitud
0,Ankkalampi Kruununhaka,60.171618,24.955742
1,Contextias plommonet,60.173491,24.958027
2,Marias Asyl,60.165295,24.943598
3,Temppeliaukion kirkon takkahuone,60.172804,24.926160
4,Entropy: Tarha,60.170142,24.922785


In [43]:
import geopandas as gpd

In [44]:
gdf = gpd.GeoDataFrame(df_nur, geometry = gpd.points_from_xy(df_nur.longitud,df_nur.latitud))
gdf.head()

,name,latitud,longitud,geometry
0,Ankkalampi Kruununhaka,60.171618,24.955742,POINT (24.95574 60.17162)
1,Contextias plommonet,60.173491,24.958027,POINT (24.95803 60.17349)
2,Marias Asyl,60.165295,24.943598,POINT (24.94360 60.16529)
3,Temppeliaukion kirkon takkahuone,60.172804,24.926160,POINT (24.92616 60.17280)
4,Entropy: Tarha,60.170142,24.922785,POINT (24.92279 60.17014)


In [45]:
from cartoframes.viz import Map, Layer, popup_element

In [46]:
Map(Layer(gdf, popup_hover = [popup_element("name","Nursery School")]))

## Tercera llamada: Nightclubs

In [47]:
params = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{emp1.get('coordinates')[1]},{emp1.get('coordinates')[0]}",
              "query": "Nightlife Spot",
              "limit": 25
         }
resp = requests.get(url=url1, params=params)
nidata = json.loads(resp.text)

In [48]:
nidata.keys()

dict_keys(['meta', 'response'])

In [49]:
decoding_data = nidata.get("response")

In [50]:
decoded = decoding_data.get("groups")[0]


In [51]:
nic = decoded.get("items")


In [52]:
mapani_nombre = ["venue","name"]
mapani_latitud = ["venue","location","lat"]
mapani_longitud = ["venue","location","lng"]

In [53]:
print(getFromDict(nic[0],mapa_nombre))
print(getFromDict(nic[0],m_latitud))
print(getFromDict(nic[0],m_longitud))

Kaisla
60.171877813403036
24.947377580647792


In [54]:
nightclubs = []
for dic in nic:
    nc = {}
    nc["name"] = getFromDict(dic,mapa_nombre)
    nc["latitud"] = getFromDict(dic,m_latitud)
    nc["longitud"] = getFromDict(dic,m_longitud)
    nightclubs.append(nc)

In [55]:
nightclubs[0]

{'name': 'Kaisla',
 'latitud': 60.171877813403036,
 'longitud': 24.947377580647792}

In [56]:
df_nc = pd.DataFrame(nightclubs)
df_nc.head()

,name,latitud,longitud
0,Kaisla,60.171878,24.947378
1,Sori Taproom,60.171963,24.947435
2,Radisson Blu Plaza Business Lounge,60.172346,24.945432
3,Champagne bar,60.168571,24.942689
4,Strindberg kirjastobaari,60.167575,24.946120


In [57]:
gdf = gpd.GeoDataFrame(df_nc, geometry = gpd.points_from_xy(df_nc.longitud,df_nc.latitud))
gdf.head()

,name,latitud,longitud,geometry
0,Kaisla,60.171878,24.947378,POINT (24.94738 60.17188)
1,Sori Taproom,60.171963,24.947435,POINT (24.94744 60.17196)
2,Radisson Blu Plaza Business Lounge,60.172346,24.945432,POINT (24.94543 60.17235)
3,Champagne bar,60.168571,24.942689,POINT (24.94269 60.16857)
4,Strindberg kirjastobaari,60.167575,24.946120,POINT (24.94612 60.16757)


In [58]:
Map(Layer(gdf, popup_hover = [popup_element("name","Night Club")]))

## Cuarta llamada: Starbucks

In [59]:
params = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{emp1.get('coordinates')[1]},{emp1.get('coordinates')[0]}",
              "query": "Starbucks",
              "limit": 5
         }
resp = requests.get(url=url1, params=params)
strdata = json.loads(resp.text)


In [60]:
strdata.keys()

dict_keys(['meta', 'response'])

In [61]:
decoding_data = strdata.get("response")

In [62]:
decoded = decoding_data.get("groups")[0]

In [63]:
star = decoded.get("items")

In [64]:
mapastar_nombre = ["venue","name"]
mapastar_latitud = ["venue","location","lat"]
mapastar_longitud = ["venue","location","lng"]

In [65]:
print(getFromDict(star[0],mapastar_nombre))
print(getFromDict(star[0],mapastar_latitud))
print(getFromDict(star[0],mapastar_longitud))

Starbucks
60.167736932209785
24.94336882366579


In [66]:
starbucks = []
for dic in star:
    st = {}
    st["name"] = getFromDict(dic,mapa_nombre)
    st["latitud"] = getFromDict(dic,m_latitud)
    st["longitud"] = getFromDict(dic,m_longitud)
    starbucks.append(st)

In [67]:
starbucks[0]

{'name': 'Starbucks',
 'latitud': 60.167736932209785,
 'longitud': 24.94336882366579}

In [68]:
df_st = pd.DataFrame(starbucks)
df_st.head()

,name,latitud,longitud
0,Starbucks,60.167737,24.943369
1,Starbucks,60.169411,24.933308
2,Starbucks,60.210526,25.079720
3,Starbucks,60.217786,24.810545
4,Starbucks,60.260766,24.854019


In [69]:
gdf4 = gpd.GeoDataFrame(df_st, geometry = gpd.points_from_xy(df_st.longitud,df_st.latitud))
gdf4.head()

,name,latitud,longitud,geometry
0,Starbucks,60.167737,24.943369,POINT (24.94337 60.16774)
1,Starbucks,60.169411,24.933308,POINT (24.93331 60.16941)
2,Starbucks,60.210526,25.079720,POINT (25.07972 60.21053)
3,Starbucks,60.217786,24.810545,POINT (24.81054 60.21779)
4,Starbucks,60.260766,24.854019,POINT (24.85402 60.26077)


In [70]:
Map(Layer(gdf4, popup_hover = [popup_element("name","Starbucks")]))

## Quinta llamada: Restaurante Vegano

In [71]:
params = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{emp1.get('coordinates')[1]},{emp1.get('coordinates')[0]}",
              "query": "Vegan Restaurant",
              "limit": 20
         }
resp = requests.get(url=url1, params=params)
vegdata = json.loads(resp.text)


In [72]:
decoding_data = vegdata.get("response")

In [73]:
decoded = decoding_data.get("groups")[0]

In [74]:
vegg = decoded.get("items")

In [75]:
mapavegg_nombre = ["venue","name"]
mapavegg_latitud = ["venue","location","lat"]
mapavegg_longitud = ["venue","location","lng"]

In [76]:
print(getFromDict(vegg[0],mapavegg_nombre))
print(getFromDict(vegg[0],mapavegg_latitud))
print(getFromDict(vegg[0],mapavegg_longitud))

Zucchini
60.16616033742283
24.949550935785922


In [77]:
vegano = []
for dic in vegg:
    vg = {}
    vg["name"] = getFromDict(dic,mapavegg_nombre)
    vg["latitud"] = getFromDict(dic,mapavegg_latitud)
    vg["longitud"] = getFromDict(dic,mapavegg_longitud)
    vegano.append(vg)

In [78]:
vegano[0]

{'name': 'Zucchini',
 'latitud': 60.16616033742283,
 'longitud': 24.949550935785922}

In [79]:
df_vg = pd.DataFrame(vegano)
df_vg.head()

,name,latitud,longitud
0,Zucchini,60.166160,24.949551
1,Qulma,60.171590,24.956639
2,Roots Kitchen,60.179254,24.951582
3,Sen Chay,60.177595,24.950797
4,Eat Your Greens!,60.169102,24.948195


In [80]:
gdf5 = gpd.GeoDataFrame(df_vg, geometry = gpd.points_from_xy(df_vg.longitud,df_vg.latitud))
gdf5.head()

,name,latitud,longitud,geometry
0,Zucchini,60.166160,24.949551,POINT (24.94955 60.16616)
1,Qulma,60.171590,24.956639,POINT (24.95664 60.17159)
2,Roots Kitchen,60.179254,24.951582,POINT (24.95158 60.17925)
3,Sen Chay,60.177595,24.950797,POINT (24.95080 60.17759)
4,Eat Your Greens!,60.169102,24.948195,POINT (24.94820 60.16910)


In [81]:
Map(Layer(gdf4, popup_hover = [popup_element("name","Venga Restaurant")]))

In [82]:
df_vg.head()

,name,latitud,longitud,geometry
0,Zucchini,60.166160,24.949551,POINT (24.94955 60.16616)
1,Qulma,60.171590,24.956639,POINT (24.95664 60.17159)
2,Roots Kitchen,60.179254,24.951582,POINT (24.95158 60.17925)
3,Sen Chay,60.177595,24.950797,POINT (24.95080 60.17759)
4,Eat Your Greens!,60.169102,24.948195,POINT (24.94820 60.16910)


In [83]:
suomi

,city,country_code,latitude,longitude,name,category_code,number_of_employees,founded_year
0,Helsinki,FIN,60.171821,24.945167,Apaja,games_video,44,2001


In [84]:
from haversine import haversine

for i, j in suomi.iterrows():
    emp1 = [suomi["latitude"], suomi["longitude"]]
    print(emp1)
    
            

[0    60.171821
Name: latitude, dtype: float64, 0    24.945167
Name: longitude, dtype: float64]


In [85]:
for a, b in suomi.iterrows():
    emp1 = [suomi["latitude"], suomi["longitude"]]
    print(emp1)

[0    60.171821
Name: latitude, dtype: float64, 0    24.945167
Name: longitude, dtype: float64]


In [86]:
empresa = pd.DataFrame([[60.171821, 24.945167 ], [60.171821, 24.945167 ], [60.171821, 24.945167 ]], columns = ["latitude", "longitude"])

In [87]:
empresa

,latitude,longitude
0,60.171821,24.945167
1,60.171821,24.945167
2,60.171821,24.945167


In [88]:
for p, h in empresa.iterrows():
    emp1 = [empresa["latitude"], empresa["longitude"]]
    print(df_vg.apply(lambda row: haversine(60.171821, 24.945167, [row["latitud"],row["longitud"]]), axis=1))

TypeError: cannot unpack non-iterable float object

In [ ]:
list(emp1)

In [ ]:
type(df_vg)

In [ ]:
for p in empresa.iterrows():
    emp1 = [empresa["latitude"], empresa["longitude"]]
    print(df_vg.apply(lambda row: haversine(60.171821, 24.945167, [row["latitud"],row["longitud"]]), axis=1))